# Prepare Data for food and nutrients

## Imports and Data

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

import sqlite3
import os

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

In [81]:
food_df = pd.read_csv('../data/FoodData/food.csv')

In [87]:
food_df.shape

(78026, 5)

In [60]:
food_df.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
1,319875,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
2,319876,market_acquisition,"HUMMUS, SABRA CLASSIC",16.0,2019-04-01
3,319877,sub_sample_food,Hummus,16.0,2019-04-01
4,319878,sub_sample_food,Hummus,16.0,2019-04-01


# Let's clean the food.csv dataset

To get rid of the duplicated rows, first we must normalize the names so that we can see if names are duplicated. To start, lets make it all lowercase.

In [110]:
import re

def normalize_description(s):
    if pd.isna(s):
        return s

    # lowercase
    s = s.lower()

    # keep text before dash
    s = s.split('-')[0]

    # remove punctuation
    s = re.sub(r'[^\w\s]', '', s)

    # normalize whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

In [111]:
food_df_normal = food_df.copy()

food_df_normal['description'] = food_df['description'].apply(normalize_description)

In [112]:
food_df_normal.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,hummus sabra classic,16.0,2019-04-01
1,319875,market_acquisition,hummus sabra classic,16.0,2019-04-01
2,319876,market_acquisition,hummus sabra classic,16.0,2019-04-01
3,319877,sub_sample_food,hummus,16.0,2019-04-01
4,319878,sub_sample_food,hummus,16.0,2019-04-01


### Remove duplicated rows

We have a lot of duplicated rows. The majority of rows are duplicated.

In [54]:
food_df_normal.loc[food_df_normal['description'].duplicated()]

,fdc_id,data_type,description,food_category_id,publication_date
1,319875,market_acquisition,hummus sabra classic,16.0,2019-04-01
2,319876,market_acquisition,hummus sabra classic,16.0,2019-04-01
4,319878,sub_sample_food,hummus,16.0,2019-04-01
5,319879,sample_food,hummus sabra classic,16.0,2019-04-01
6,319880,market_acquisition,hummus sabra classic,16.0,2019-04-01
...,...,...,...,...,...
78021,2751499,sub_sample_food,shallots bulb peeled root removed raw,NaN,2025-07-03
78022,2751500,sub_sample_food,shallots bulb peeled root removed raw,NaN,2025-07-03
78023,2751501,sub_sample_food,shallots bulb peeled root removed raw,NaN,2025-07-03
78024,2751502,sub_sample_food,shallots bulb peeled root removed raw,NaN,2025-07-03


Ok, so now we need to remove the duplicated rows, but we need to be strategic about which to throw away. My initial attempt:  
<code> food_df_clean = food_df_normal.drop_duplicates(subset=['description'], keep='last') </code>  
Worked, however this issue was that a lot of the nutritional info I wanted was not provided.  

To deal with this, I will first search thru the duplicated rows to see which one provides the most nutritional info and keep that one. 

In [113]:
# Let's make a dataframe of just duplicated data

food_df_duplicated_total = food_df_normal.loc[food_df_normal['description'].duplicated()]

In [85]:
# Now we will make a dataframe of one entry from each item that was duplicated. 
# The duplicated item will later be replaced by the one with the most nutrients. 

# Find all duplicated descriptions
duplicated_names = food_df_normal[food_df_normal.duplicated(subset=['description'], keep=False)]

# Keep only one entry per duplicated description (first occurrence for now)
food_df_duplicated = duplicated_names.drop_duplicates(subset=['description'], keep='first')

In [57]:
food_df_duplicated_total.head()

,fdc_id,data_type,description,food_category_id,publication_date
1,319875,market_acquisition,hummus sabra classic,16.0,2019-04-01
2,319876,market_acquisition,hummus sabra classic,16.0,2019-04-01
4,319878,sub_sample_food,hummus,16.0,2019-04-01
5,319879,sample_food,hummus sabra classic,16.0,2019-04-01
6,319880,market_acquisition,hummus sabra classic,16.0,2019-04-01


In [69]:
food_df_duplicated_total.shape

(70536, 5)

In [67]:
food_df_duplicated.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,hummus sabra classic,16.0,2019-04-01
3,319877,sub_sample_food,hummus,16.0,2019-04-01
20,319894,sample_food,hummus other,16.0,2019-04-01
84,319958,sample_food,hummus tribe classic,16.0,2019-04-01
150,320025,sample_food,milk 2 wave 22e,1.0,2019-04-01


In [70]:
food_df_duplicated.shape

(1417, 5)

Ok, now we have 2 dataframes, food_df_duplicated_total has every duplicated row, and food_df_duplicated has one row for each duplicated row.  

Now, for each row in food_df_duplicated, we will search through the rowsn in food_df_duplicated_total to find the fdc_id that has the most nutritional information and replace that entry in food_df_duplicated.

In [86]:
# To do this, we first need nutrient_df. Let's import it. 

nutrient_df = pd.read_csv("FoodData/nutrient.csv")

In [17]:
nutrient_df.shape

(477, 5)

In [18]:
nutrient_df.head()

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0


In [87]:
# We also need food_nutrient_df to convert the values.

food_nutrient_df = pd.read_csv("FoodData/food_nutrient.csv", usecols=[0,1,2,3])

In [27]:
food_nutrient_df.shape

(159285, 4)

In [28]:
food_nutrient_df.head()

,id,fdc_id,nutrient_id,amount
0,2201847,319877,1051,56.30
1,2201845,319877,1002,1.28
2,2201846,319877,1004,19.00
3,2201844,319877,1007,1.98
4,2201852,319878,1091,188.00


### Now that we have our dataframes, lets go over the logic

lets look up the nutrient info of the first entry in <code>food_df_duplicated</code> to get a feel for how to do this.

In [93]:
# First find the descritption and fdc_id of the fist item.

d0 = food_df_duplicated.loc[0, 'description']
print(d1)

fdc0 = food_df_duplicated.loc[0, 'fdc_id']
print(fdc1)

hummus sabra classic
319874


In [107]:
# Now we use that fdc_id to get all nutrient id's associated with it
# The dataframe is empty, meaning this humman sabra classic has no nutritional information 

fn0 = food_nutrient_df[food_nutrient_df['fdc_id']==fdc0] 
fn0.head()

,id,fdc_id,nutrient_id,amount


In [106]:
# Let's check for another item to see what we get if there is nutritional info

d3 = food_df_duplicated.loc[3, 'description']
print(d3)

fdc3 = food_df_duplicated.loc[3, 'fdc_id']
print(fdc3)

hummus
319877


In [103]:
# Now we use that fdc_id to get all nutrient id's associated with it

fn3 = food_nutrient_df[food_nutrient_df['fdc_id']==fdc3] 
fn3.head()

,id,fdc_id,nutrient_id,amount
0,2201847,319877,1051,56.30
1,2201845,319877,1002,1.28
2,2201846,319877,1004,19.00
3,2201844,319877,1007,1.98


When we have this dataframe, we can check if it lists the desired nutritient_id's, which are listed below. 

In [117]:
# These are the nutrients we want to show for each food and recipe

nutri = {'calories': 1008, 'protein_g': 1003, 'carbs_g': 1005, 'fat_g': 1004, 'fiber_g': 1079, 'sodium_mg': 1093}
nutri_iter = ['calories', 'protein_g', 'carbs_g', 'fat_g', 'fiber_g', 'sodium_mg']

In [122]:
present_nutrients = set(fn3['nutrient_id'])

In [123]:
score = sum(nid in present_nutrients for nid in nutri.values())
print("Nutrient completeness score:", score)

Nutrient completeness score: 1


Now we want to do this for every duplicated item until we get a score of 6, or keep the highest score if none have a score of 6.

In [125]:
duplicated_hummas = food_df_duplicated_total[food_df_duplicated_total['description']==d3]
duplicated_hummas.shape

(87, 5)

In [131]:
duplicated_hummas.head()

,fdc_id,data_type,description,food_category_id,publication_date
4,319878,sub_sample_food,hummus,16.0,2019-04-01
8,319882,sub_sample_food,hummus,16.0,2019-04-01
9,319883,sub_sample_food,hummus,16.0,2019-04-01
10,319884,sub_sample_food,hummus,16.0,2019-04-01
18,319892,sub_sample_food,hummus,16.0,2019-04-01


In [141]:
max_nutri = 0
best_fdc_id = None
i = 0
nutri_id_to_name = {v: k for k, v in nutri.items()}

while max_nutri < 6 and i < duplicated_hummas.shape[0]:
    fdc = duplicated_hummas.iloc[i]['fdc_id']

    # Get nutrients for this fdc_id
    fn = food_nutrient_df[food_nutrient_df['fdc_id'] == fdc]

    present_nutrients = set(fn['nutrient_id'])
    score = sum(nid in present_nutrients for nid in nutri.values())

    present_nutrients = sorted(set(fn3['nutrient_id']))

    name = ""
    
    for nid in present_nutrients:
        name += nutri_id_to_name.get(nid, f"other ({nid}), ")

    if i==30:
        name+="suprise!"
    
    print(f"Row {i} | fdc_id={fdc} | score={score} | nutrients: {name}")

    # Update best score
    if score > max_nutri:
        max_nutri = score
        best_fdc_id = fdc

    # Early exit if perfect match
    if score == 6:
        break

    i += 1

Row 0 | fdc_id=319878 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 1 | fdc_id=319882 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 2 | fdc_id=319883 | score=0 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 3 | fdc_id=319884 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 4 | fdc_id=319892 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 5 | fdc_id=319893 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 6 | fdc_id=319899 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 7 | fdc_id=319900 | score=1 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 8 | fdc_id=319906 | score=0 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 9 | fdc_id=319907 | score=0 | nutrients: other (1002), fat_gother (1007), other (1051), 
Row 10 | fdc_id=319908 | score=1 | nutrients: other (1002), fat_gother

hmmm.. it seems that they all have the same amount of nutrients. In that case, this will be a little trickier. 

### Initial Attempt

In [88]:
food_df_clean = food_df_normal.drop_duplicates(subset=['description'], keep='last')

In [94]:
food_df_clean.head(30)

,fdc_id,data_type,description,food_category_id,publication_date
83,319957,sub_sample_food,hummus sabra classic,16.0,2019-04-01
130,320004,sub_sample_food,hummus tribe classic,16.0,2019-04-01
148,320022,sub_sample_food,hummus,16.0,2019-04-01
149,320023,sub_sample_food,hummus other,16.0,2019-04-01
474,320349,sub_sample_food,milk 2,1.0,2019-04-01
476,320351,sub_sample_food,milk 2 wave 22e,1.0,2019-04-01
477,320353,sample_food,beef eye of round roaststeak lean only raw ani...,13.0,2019-04-01
480,320356,sub_sample_food,proximates beef eye of round roaststeak lean o...,13.0,2019-04-01
481,320357,sample_food,beef eye of round roaststeak select raw comp13...,13.0,2019-04-01
492,320368,market_acquisition,beef eye of round roast raw er37,13.0,2019-04-01


In [95]:
food_df_clean.shape

(7490, 5)

In [113]:
# save the cleaned df to a csv file
food_df_clean.to_csv("FoodData/food_clean.csv", index=False)

# Create Database

Now that we have the tables that we want to work with, we can make a database file using SQLight to help us reference the information.

In [6]:
conn = sqlite3.connect("food_data.db")
cur = conn.cursor()

### Create dataframes for the tables we want to use

In [146]:
food_df_clean = pd.read_csv("FoodData/food_clean.csv")

In [171]:
nutrient_df = pd.read_csv("FoodData/nutrient.csv")

nutrient_df = nutrient_df.rename(columns={"id": "nutrient_id"})

In [24]:
food_nutrient_df = pd.read_csv("FoodData/food_nutrient.csv", usecols=[0,1,2,3])

food_nutrient_df = food_nutrient_df.rename(columns={"id": "fdc_nutrient_id"})

In [176]:
print("Food table:", food_df_clean.shape)
print("Nutrient table:", nutrient_df.shape)
print("Food-Nutrient table:", food_nutrient_df.shape)

Food table: (7490, 5)
Nutrient table: (477, 5)
Food-Nutrient table: (159285, 4)


### Let's add the food table to our database

In [177]:
food_df_clean.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319957,sub_sample_food,hummus sabra classic,16.0,2019-04-01
1,320004,sub_sample_food,hummus tribe classic,16.0,2019-04-01
2,320022,sub_sample_food,hummus,16.0,2019-04-01
3,320023,sub_sample_food,hummus other,16.0,2019-04-01
4,320349,sub_sample_food,milk 2,1.0,2019-04-01


In [159]:
# conn.execute("DROP TABLE food;")

In [158]:
conn.execute("""
CREATE TABLE food (
    fdc_id INTEGER PRIMARY KEY,
    data_type TEXT,
    description TEXT,
    food_category_id INTEGER,
    publication_date TEXT
);
""")

In [160]:
food_df_clean.to_sql("food", conn, if_exists="append", index=False)

7490

In [178]:
pd.read_sql("SELECT * FROM food LIMIT 5;", conn)

,fdc_id,data_type,description,food_category_id,publication_date
0,319957,sub_sample_food,hummus sabra classic,16,2019-04-01
1,320004,sub_sample_food,hummus tribe classic,16,2019-04-01
2,320022,sub_sample_food,hummus,16,2019-04-01
3,320023,sub_sample_food,hummus other,16,2019-04-01
4,320349,sub_sample_food,milk 2,1,2019-04-01


### Let's add the nutrient table to our database

In [179]:
nutrient_df.head()

,nutrient_id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0


In [181]:
# conn.execute("DROP TABLE nutrient")

In [182]:
conn.execute("""
CREATE TABLE nutrient (
    nutrient_id INTEGER PRIMARY KEY,
    name TEXT,
    unit_name TEXT,
    nutrient_nbr INTEGER,
    rank INTEGER
);
""")

In [183]:
nutrient_df.to_sql("nutrient", conn, if_exists="append", index=False)

477

In [184]:
pd.read_sql("SELECT * FROM nutrient LIMIT 5;", conn)

,nutrient_id,name,unit_name,nutrient_nbr,rank
0,1001,Solids,G,201,200
1,1002,Nitrogen,G,202,500
2,1003,Protein,G,203,600
3,1004,Total lipid (fat),G,204,800
4,1005,"Carbohydrate, by difference",G,205,1110


### Finally we will add the food_nutrient table to our dataframe

In [185]:
food_nutrient_df.head()

,fdc_nutrient_id,fdc_id,nutrient_id,amount
0,2201847,319877,1051,56.30
1,2201845,319877,1002,1.28
2,2201846,319877,1004,19.00
3,2201844,319877,1007,1.98
4,2201852,319878,1091,188.00


In [187]:
# conn.execute("DROP TABLE food_nutrient;")

In [188]:
conn.execute("""
CREATE TABLE food_nutrient (
    fdc_nutrient_id INTEGER PRIMARY KEY,
    fdc_id INTEGER,
    nutrient_id INTEGER,
    amount REAL,
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (nutrient_id) REFERENCES nutrient(nutrient_id)
);
""")

In [189]:
food_nutrient_df.to_sql("food_nutrient", conn, if_exists="append", index=False)

159285

In [190]:
pd.read_sql("SELECT * FROM food_nutrient LIMIT 5;", conn)

,fdc_nutrient_id,fdc_id,nutrient_id,amount
0,2201844,319877,1007,1.98
1,2201845,319877,1002,1.28
2,2201846,319877,1004,19.00
3,2201847,319877,1051,56.30
4,2201848,319878,1087,46.00


# Working with the data

### Now that we have our tables we are ready to start working with the data

In [39]:
conn = sqlite3.connect("food_data.db")
cur = conn.cursor()

In [40]:
# These are the nutrients we want to show for each food and recipe

nutri = {'calories': 1008, 'protein_g': 1003, 'carbs_g': 1005, 'fat_g': 1004, 'fiber_g': 1079, 'sodium_mg': 1093}
nutri_iter = ['calories', 'protein_g', 'carbs_g', 'fat_g', 'fiber_g', 'sodium_mg']

In [41]:
print([f"{nutrien}: {nutri[nutrien]}" for nutrien in nutri_iter])

['calories: 1008', 'protein_g: 1003', 'carbs_g: 1005', 'fat_g: 1004', 'fiber_g: 1079', 'sodium_mg: 1093']


In [44]:
# LEFT JOIN

food_to_calories_query = f"""
SELECT f.description AS "Food Item", 
calories.amount AS Calories, 
protein_g.amount AS "Protein in Grams", 
carbs_g.amount AS "Carbs in Grams", 
fat_g.amount AS "Fat in Grams", 
fiber_g.amount AS "Fiber in Grams", 
sodium_mg.amount AS "Sodium in Milligrams"

FROM food f

LEFT JOIN food_nutrient calories
    ON f.fdc_id = calories.fdc_id
    AND calories.nutrient_id = {nutri['calories']}

LEFT JOIN food_nutrient protein_g
    ON f.fdc_id = protein_g.fdc_id
    AND protein_g.nutrient_id = {nutri['protein_g']}

LEFT JOIN food_nutrient carbs_g
    ON f.fdc_id = carbs_g.fdc_id
    AND carbs_g.nutrient_id = {nutri['carbs_g']}

LEFT JOIN food_nutrient fat_g
    ON f.fdc_id = fat_g.fdc_id
    AND fat_g.nutrient_id = {nutri['fat_g']}

LEFT JOIN food_nutrient fiber_g
    ON f.fdc_id = fiber_g.fdc_id
    AND fiber_g.nutrient_id = {nutri['fiber_g']}

LEFT JOIN food_nutrient sodium_mg
    ON f.fdc_id = sodium_mg.fdc_id
    AND sodium_mg.nutrient_id = {nutri['sodium_mg']}

"""

In [45]:
pd.read_sql(food_to_calories_query, conn)

,Food Item,Calories,Protein in Grams,Carbs in Grams,Fat in Grams,Fiber in Grams,Sodium in Milligrams
0,hummus sabra classic,NaN,NaN,NaN,NaN,NaN,NaN
1,hummus tribe classic,NaN,NaN,NaN,NaN,NaN,NaN
2,hummus,NaN,NaN,NaN,NaN,NaN,NaN
3,hummus other,NaN,NaN,NaN,NaN,NaN,NaN
4,milk 2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
7485,tomatillos dehusked raw,NaN,NaN,NaN,NaN,1.6,NaN
7486,cabbage napa leaf destemmed raw,NaN,NaN,NaN,NaN,0.7,NaN
7487,leeks bulb and greens root removed raw,NaN,NaN,NaN,NaN,3.0,NaN
7488,green onion scallion bulb and greens root remo...,NaN,NaN,NaN,NaN,3.0,NaN


At this point, something ive noticed is that a lot of nutrition info is missing and the likely reason for this is because i choose the last item if items had the same name and instead I need a better aproach in order to select the version that has the most nutrition info and that has info for most of my nutritional info.  

My idea for an apporach is to first create an array of duplicated items based on identified names. Then from that list I will search thru and compare each entrie's food_nutrition_id and see how many of the desired nutritional info it has and give it a number based on that. If that number is higher than the previous entry ill keep that in my final entries array. Once I get all the nutritional info I need or I run out of entrues, I will keep that entry in my final entries array. My final entries array will keep one row signified by its fdc_id from all the duplicated arrays. then finally I will update my food_df_clean to first delete all duplicated rows, keeping none, then adding the rows from my final entries array. This way I will have as much info as possible. Then I will add this new df back to my database and run my query again, hopefully having full rows of health info-data. 